The tptp version

Axiom of choice
universes
Forcing
Well founded recursion

Open Sets are semdecidable.
What is an open set anyway? We draw a blob without a boundary. Open sets are algerbaically defined as a collection being closed under countable interesection and arbitrary union.
Escardo synthetiuc topology blog posts and paper
Thew ideas in Marshall and MarshallB


realizibaility "Associate to a proposition a set of programs in your favorite programing language, an element of that set is a realizer of the proposition"
A proof of two normalizable terms being equal is not needed.
sat(p) has the assignment as proof. sure. Let's say p are ground smt formula 
sat(?p) -> sat(?q) is a prcoesure to turn the satisying assignemt of p into one of q
forall q,   (for any propositional formula q)
exists p, P(p), there is propositional

There are slides on smt somewhere about equisatisfiable and other things.
Is there some example where the intuiitionistic character would come into play

atom = min c^T x st Ax >= b.  Proof = dual vector.

examples and counterexamples.

"The motivation behind the definition of (ϕ → ψ) D given by G¨odel is as follows: from a witness x to the hypothesis ϕD one should be able to obtain a witness u to the conclusion ψ D , such that from a counterexample v to the conclusion one should be able to find a counterexample y to the hypothesis"

forall x, exists y, phi(x,y) -> exists Y forall x phi(x, Y @ x) axiom of choice is an internalized skolemization

blame calculus
exists forall problems are function syunthesis problems. Running dialectica bnackwards?



Comprehension is a relfection principle from logic to objects.  If we weanted to reflect substercutrual loigc , we would not gert sets. We'd get multisets. Similar perhaps if we want a "set theory" that takes some other entity as primitive, like lists or sequences.

Model theory and databases. Query containment using TPTP solvers.


domain theory in python



In [ ]:
%%file /tmp/systemt.p

thf( k , type, ) % constant
thf( recurs, type, )
thf( )

k(S,T) = S
s(R,S,T) = r(T) @ s(T)


r(F,G,0) = F
r(F,G,succ(N)) = G @ N @ r(F,G,N)